In [38]:
# Imports: regex utilities, pandas, and pathlib for file paths
import re
import pandas as pd
from pathlib import Path

In [39]:
# Source Excel file path for state/territory vacancies
file_path = Path('../data/Actual/raw/states_territories.xlsx')

In [40]:
# Load the raw worksheet into a DataFrame
#   - sheet_name: source tab
#   - header: first row contains column names
df = pd.read_excel(file_path, sheet_name="Data1", header=0)

In [41]:
# Quick peek at the first 15 rows to inspect header/data lines
df.head(15)

,Unnamed: 0,Job Vacancies ; New South Wales ;,Job Vacancies ; Victoria ;,Job Vacancies ; Queensland ;,Job Vacancies ; South Australia ;,Job Vacancies ; Western Australia ;,Job Vacancies ; Tasmania ;,Job Vacancies ; Northern Territory ;,Job Vacancies ; Australian Capital Territory ;,Job Vacancies ; Australia ;,...,Job Vacancies ; Australia ;.2,Standard Error of Job Vacancies ; New South Wales ;,Standard Error of Job Vacancies ; Victoria ;,Standard Error of Job Vacancies ; Queensland ;,Standard Error of Job Vacancies ; South Australia ;,Standard Error of Job Vacancies ; Western Australia ;,Standard Error of Job Vacancies ; Tasmania ;,Standard Error of Job Vacancies ; Northern Territory ;,Standard Error of Job Vacancies ; Australian Capital Territory ;,Standard Error of Job Vacancies ; Australia ;
0,Unit,000,000,000,000,000,000,000,000,000,...,000,000,000,000,000,000,000,000,000,000
1,Series Type,Original,Original,Original,Original,Original,Original,Original,Original,Original,...,Trend,Original,Original,Original,Original,Original,Original,Original,Original,Original
2,Data Type,STOCK,STOCK,STOCK,STOCK,STOCK,STOCK,STOCK,STOCK,STOCK,...,STOCK,RATIO,RATIO,RATIO,RATIO,RATIO,RATIO,RATIO,RATIO,RATIO
3,Frequency,Quarter,Quarter,Quarter,Quarter,Quarter,Quarter,Quarter,Quarter,Quarter,...,Quarter,Quarter,Quarter,Quarter,Quarter,Quarter,Quarter,Quarter,Quarter,Quarter
4,Collection Month,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
5,Series Start,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1979-05-15 00:00:00,...,1979-05-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1983-11-15 00:00:00,1979-05-15 00:00:00
6,Series End,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,...,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00,2025-05-15 00:00:00
7,No. Obs,167,167,167,167,167,167,167,167,185,...,185,167,167,167,167,167,167,167,167,185
8,Series ID,A590714V,A590732X,A590750C,A590768A,A590786F,A590804X,A590822C,A590840J,A590696A,...,A590700F,A590715W,A590733A,A590751F,A590769C,A590787J,A590805A,A590823F,A590841K,A590697C
9,1979-05-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.9,...,42.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# Drop the first 9 rows (metadata in the sheet) and reset the index
df = df.iloc[9:].reset_index(drop=True)

In [43]:
# Inspect column names, dtypes, and non-null counts after trimming header rows
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 21 columns):
 #   Column                                                             Non-Null Count  Dtype 
---  ------                                                             --------------  ----- 
 0   Unnamed: 0                                                         185 non-null    object
 1   Job Vacancies ;  New South Wales ;                                 162 non-null    object
 2   Job Vacancies ;  Victoria ;                                        162 non-null    object
 3   Job Vacancies ;  Queensland ;                                      162 non-null    object
 4   Job Vacancies ;  South Australia ;                                 162 non-null    object
 5   Job Vacancies ;  Western Australia ;                               162 non-null    object
 6   Job Vacancies ;  Tasmania ;                                        162 non-null    object
 7   Job Vacancies ;  Northern Territory

In [44]:
# Rename the first column to a canonical "Date" and parse as datetime
time_col = df.columns[0]
df = df.rename(columns={time_col: "Date"})
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
# Validate that "Date" is datetime and review missingness after parsing
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 21 columns):
 #   Column                                                             Non-Null Count  Dtype         
---  ------                                                             --------------  -----         
 0   Date                                                               185 non-null    datetime64[ns]
 1   Job Vacancies ;  New South Wales ;                                 162 non-null    object        
 2   Job Vacancies ;  Victoria ;                                        162 non-null    object        
 3   Job Vacancies ;  Queensland ;                                      162 non-null    object        
 4   Job Vacancies ;  South Australia ;                                 162 non-null    object        
 5   Job Vacancies ;  Western Australia ;                               162 non-null    object        
 6   Job Vacancies ;  Tasmania ;                                       

In [45]:
# Clean column names into consistent, concise identifiers
#  - Standard errors -> prefix with "SE_" and suffix "_Total"
#  - Job Vacancies -> "<Region>_Total"; handle Trend/Seasonal special cases
#  - Otherwise keep name unchanged
# This simplifies downstream selection and plotting.
def clean_col(name: str):
    name = str(name).strip()
    if name.startswith("Standard Error of Job Vacancies"):
        m = re.search(r"Standard Error of Job Vacancies\s*;\s*(.*?)\s*;", name)
        state = m.group(1) if m else name
        return f"SE_{state}_Total"
    if name.startswith("Job Vacancies"):
        if "Seasonally Adjusted" in name:
            return "Australia_Total_Seasonal"
        if "Trend" in name:
            return "Australia_Total_Trend"
        m = re.search(r"Job Vacancies\s*;\s*(.*?)\s*;", name)
        state = m.group(1) if m else name
        return f"{state}_Total"
    return name

# Apply the cleaner and inspect the new schema
df = df.rename(columns={c: clean_col(c) for c in df.columns})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Date                                   185 non-null    datetime64[ns]
 1   New South Wales_Total                  162 non-null    object        
 2   Victoria_Total                         162 non-null    object        
 3   Queensland_Total                       162 non-null    object        
 4   South Australia_Total                  162 non-null    object        
 5   Western Australia_Total                162 non-null    object        
 6   Tasmania_Total                         162 non-null    object        
 7   Northern Territory_Total               162 non-null    object        
 8   Australian Capital Territory_Total     162 non-null    object        
 9   Australia_Total                        180 non-null    object    

In [46]:
# Remove any duplicated columns that resulted from the raw sheet structure
# Keep the first occurrence and copy to avoid chained-assignment warnings
df = df.loc[:, ~df.columns.duplicated()].copy()
# Verify final columns and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Date                                   185 non-null    datetime64[ns]
 1   New South Wales_Total                  162 non-null    object        
 2   Victoria_Total                         162 non-null    object        
 3   Queensland_Total                       162 non-null    object        
 4   South Australia_Total                  162 non-null    object        
 5   Western Australia_Total                162 non-null    object        
 6   Tasmania_Total                         162 non-null    object        
 7   Northern Territory_Total               162 non-null    object        
 8   Australian Capital Territory_Total     162 non-null    object        
 9   Australia_Total                        180 non-null    object    

In [47]:
# Filter to the analysis period starting Nov 1983 and reset the index
df = df[df["Date"] >= "1983-11-01"].reset_index(drop=True)
# Sanity-check after filtering and review schema
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167 entries, 0 to 166
Data columns (total 19 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Date                                   167 non-null    datetime64[ns]
 1   New South Wales_Total                  162 non-null    object        
 2   Victoria_Total                         162 non-null    object        
 3   Queensland_Total                       162 non-null    object        
 4   South Australia_Total                  162 non-null    object        
 5   Western Australia_Total                162 non-null    object        
 6   Tasmania_Total                         162 non-null    object        
 7   Northern Territory_Total               162 non-null    object        
 8   Australian Capital Territory_Total     162 non-null    object        
 9   Australia_Total                        162 non-null    object    

In [48]:
# Split into two DataFrames:
#  - vac_df: vacancy values (non-SE columns)
#  - se_df: standard error columns with Date
se_cols = [c for c in df.columns if c.startswith("SE_")]
vac_cols =  [c for c in df.columns if c not in se_cols and c != "Series ID" and c != "Unit"]
vac_df = df[vac_cols].copy()
se_df = pd.concat([df[["Date"]], df[se_cols]], axis=1)

In [49]:
# Convert numeric-like columns from object to float for both frames
for frame in (vac_df, se_df):
    num_cols = frame.columns.drop("Date")
    for col in num_cols:
        frame[col] = pd.to_numeric(frame[col], errors="coerce")

In [50]:
# Save cleaned wide-form datasets to disk
vac_path = "../data/actual/preprocessed/state_vacancies_clean.csv"
se_path = "../data/actual/preprocessed/state_vacancies_standard_error.csv"
vac_df.to_csv(vac_path, index=False)
se_df.to_csv(se_path, index=False)

In [51]:
# Convert wide-form vacancies into a tidy long-form table
vac_long = vac_df.melt(id_vars="Date", var_name="Region", value_name="Vacancies_thousands")
# Persist long-form to CSV for downstream viz/analysis
vac_long_path = "../data/actual/preprocessed/state_vacancies_clean_long.csv"
vac_long.to_csv(vac_long_path, index=False)

In [52]:
# Reload the saved CSVs to verify outputs on disk
#  - data: wide-form vacancies
#  - se_data: standard errors (wide-form)
#  - data3: long-form vacancies
data = pd.read_csv(vac_path)
se_data = pd.read_csv(se_path)
data3 = pd.read_csv(vac_long_path)

In [53]:
# Baseline missingness summary (counts per column) before interpolation
data.isna().sum()

Date                                  0
New South Wales_Total                 5
Victoria_Total                        5
Queensland_Total                      5
South Australia_Total                 5
Western Australia_Total               5
Tasmania_Total                        5
Northern Territory_Total              5
Australian Capital Territory_Total    5
Australia_Total                       5
dtype: int64

In [54]:
# Column we require to be non-null while others are all null
keep_col = "Date"

# Build a boolean mask for rows where:
#  - every column EXCEPT `keep_col` is null
#  - and `keep_col` itself is non-null
mask = data.loc[:, data.columns != keep_col].isna().all(axis=1) & data[keep_col].notna()

# Preview the rows that match this condition (view only, no mutation)
rows = data.loc[mask]
rows

,Date,New South Wales_Total,Victoria_Total,Queensland_Total,South Australia_Total,Western Australia_Total,Tasmania_Total,Northern Territory_Total,Australian Capital Territory_Total,Australia_Total
99,2008-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,2008-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,2009-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,2009-05-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,2009-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# Reuse the mask to identify rows to drop
mask = data.loc[:, data.columns != keep_col].isna().all(axis=1) & df[keep_col].notna()

# Count rows that satisfy the condition (for logging)
num_to_drop = int(mask.sum())

# Keep only rows that DO NOT match the mask (this drops the flagged rows)
data = data.loc[~mask].copy()

print(f"Dropped {num_to_drop} rows; new shape: {df.shape}")

Dropped 5 rows; new shape: (167, 19)


In [56]:
# Baseline missingness summary (counts per column) 
data.isna().sum()

Date                                  0
New South Wales_Total                 0
Victoria_Total                        0
Queensland_Total                      0
South Australia_Total                 0
Western Australia_Total               0
Tasmania_Total                        0
Northern Territory_Total              0
Australian Capital Territory_Total    0
Australia_Total                       0
dtype: int64

In [57]:
data.to_csv(vac_path, index=False)